In [24]:
import pandas as pd
import pickle
import numpy as np
from gower import gower_matrix
from sklearn.neighbors import KNeighborsRegressor
from sklearn.utils import resample

In [3]:
data = pd.read_csv('./preprocessing/Clean_data.csv')

data.drop(columns=["Unnamed: 0", "id"], inplace=True)
data["Locality"] = data["Locality"].astype("str")
data["Fully equipped kitchen"] = data["Fully equipped kitchen"].astype("str")
data["Fireplace"] = data["Fireplace"].astype("str")
data["Terrace"] = data["Terrace"].astype("str")
data["Garden"] = data["Garden"].astype("str")
data["Swimming pool"] = data["Swimming pool"].astype("str")
data["Furnished"] = data["Furnished"].astype("str")
data["Number of rooms"] = data["Number of rooms"].astype("int64")
data["Number of facades"] = data["Number of facades"].astype("int64")

# Load INS data and tax data to link
data_insee = pd.read_csv("./preprocessing/INSEE_PostCode.csv", encoding="latin-1")
subset_columns = data_insee.columns[6:]
data_insee["PostalCodes"] = data_insee[subset_columns].apply(
    lambda row: row.dropna().tolist(), axis=1
)
data_insee.drop(columns=data_insee.columns[6:22], inplace=True)

# Import and prepare dataset from SPF Finances to join on Code INS
data_fin = pd.read_csv("./preprocessing/SPF_FIN_Stat.csv", encoding="latin-1")

# Merge two datasets on Code INS, keeping only records that are present in both tables
# Rows without postal code, or without financial data are not interesting for final join with data
data_fin_postcode = pd.merge(data_fin, data_insee, how="inner", on="Code INS")

# Unpack/explode lists of post codes to obtain dataset with one row of info per postal code
data_fin_postcode_exploded = data_fin_postcode.explode("PostalCodes")

# Convert post codes to str type and join data to original dataset
data_fin_postcode_exploded["PostalCodes"] = (
    data_fin_postcode_exploded["PostalCodes"].astype("int").astype("str")
)

In [8]:
taxdata = data_fin_postcode_exploded[['PostalCodes', 'Revenu moyen par déclaration']]
taxdata.rename(
    columns={"Revenu moyen par déclaration": "Mean_income_taxunit"},
    inplace=True,
)
taxdata

C:\Users\kvnpo\AppData\Local\Temp\ipykernel_1664\2813688415.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  taxdata.rename(


,PostalCodes,Mean_income_taxunit
0,2630,46.475
1,2000,33.144
1,2018,33.144
1,2020,33.144
1,2030,33.144
...,...,...
557,5660,30.074
558,5680,31.331
559,5620,34.111
560,5600,33.097


In [9]:
postalcodes = taxdata['PostalCodes'].to_list()
postalcodes

['2630',
 '2000',
 '2018',
 '2020',
 '2030',
 '2040',
 '2050',
 '2060',
 '2100',
 '2180',
 '2600',
 '2610',
 '2660',
 '2530',
 '2531',
 '2850',
 '2150',
 '2930',
 '2960',
 '2650',
 '2910',
 '2620',
 '2540',
 '2920',
 '2950',
 '2550',
 '2547',
 '2640',
 '2845',
 '2520',
 '2840',
 '2627',
 '2970',
 '2900',
 '2940',
 '2110',
 '2160',
 '2990',
 '2240',
 '2242',
 '2243',
 '2980',
 '2070',
 '2390',
 '2590',
 '2820',
 '2880',
 '2570',
 '2220',
 '2221',
 '2222',
 '2223',
 '2500',
 '2800',
 '2801',
 '2811',
 '2812',
 '2560',
 '2580',
 '2860',
 '2861',
 '2830',
 '2370',
 '2387',
 '2490',
 '2491',
 '2340',
 '2480',
 '2440',
 '2280',
 '2288',
 '2200',
 '2270',
 '2230',
 '2320',
 '2321',
 '2322',
 '2323',
 '2235',
 '2460',
 '2275',
 '2450',
 '2330',
 '2400',
 '2250',
 '2360',
 '2380',
 '2381',
 '2382',
 '2470',
 '2310',
 '2300',
 '2290',
 '2350',
 '2260',
 '2430',
 '2431',
 '1070',
 '1160',
 '1082',
 '1000',
 '1020',
 '1120',
 '1130',
 '1040',
 '1140',
 '1190',
 '1083',
 '1050',
 '1090',
 '1081',
 

In [4]:
data_postcodes = pd.merge(
    data_fin_postcode_exploded,
    data,
    how="inner",
    left_on="PostalCodes",
    right_on="Locality",
)

# Drop duplicate columns, rename columns
data_postcodes.drop(columns="Entités administratives_x", inplace=True)
data_postcodes.drop(columns="Locality", inplace=True)
data_postcodes.drop(columns="Code INS", inplace=True)
data_postcodes.rename(
    columns={"Entités administratives_y": "Locality"}, inplace=True
)
data_postcodes.rename(
    columns={"Nombre d'habitants": "N_Inhabitants"}, inplace=True
)
data_postcodes.rename(
    columns={"Revenu total net imposable": "Tot_taxable_income"}, inplace=True
)
data_postcodes.rename(
    columns={"Revenu moyen par déclaration": "Mean_income_taxunit"},
    inplace=True,
)
data_postcodes.rename(
    columns={"Revenu médian par déclaration": "Median_income_taxunit"},
    inplace=True,
)
data_postcodes.rename(
    columns={"Revenu moyen par habitant": "Mean_income_inhabitant"},
    inplace=True,
)
data_postcodes.rename(
    columns={"Indice de richesse": "Wealth_index"}, inplace=True
)
data_postcodes.rename(columns={"Impôt d'Etat": "State_tax"}, inplace=True)
data_postcodes.rename(
    columns={"Taxes communales et d'agglomération": "Local_tax"}, inplace=True
)
data_postcodes.rename(columns={"Impôt total": "Tot_tax"}, inplace=True)
data_postcodes.rename(columns={"Langue": "Language"}, inplace=True)
data_postcodes.rename(columns={"Région": "Region"}, inplace=True)
data_postcodes.rename(columns={"Arrondissement": "District"}, inplace=True)

# clean data types
columns_to_convert = [
    "N_Inhabitants",
    "Tot_taxable_income",
    "State_tax",
    "Local_tax",
    "Tot_tax",
]
data_postcodes[columns_to_convert] = (
    data_postcodes[columns_to_convert]
    .apply(lambda col: col.str.replace(".", "", regex=False))
    .astype(float)
)
data_postcodes["N_Inhabitants"] = data_postcodes["N_Inhabitants"].astype(int)
data_postcodes["Wealth_index"] = data_postcodes["Wealth_index"].astype(float)

# Subset data to set price range and store in attribute

subset_price_datapostcodes = data_postcodes[(data_postcodes['Price'] >= 200000) & (data_postcodes['Price'] <= 600000)]
subset_columns_datapostcodes = subset_price_datapostcodes[[
"Mean_income_taxunit",
"Subtype of property",
"State of the building",
"Surface area of the plot of land",
"Number of rooms",
"Living Area",
"Number of facades",
]]

data = subset_columns_datapostcodes
data_y = subset_price_datapostcodes['Price']

In [8]:
data

,Mean_income_taxunit,Subtype of property,State of the building,Surface area of the plot of land,Number of rooms,Living Area,Number of facades
0,46.475,APARTMENT,UNKNOWN,0.0,3,102.936296,3
1,46.475,APARTMENT,GOOD,0.0,2,54.333333,3
2,46.475,APARTMENT,GOOD,0.0,2,76.130295,3
3,46.475,HOUSE,AS_NEW,468.0,5,220.000000,2
4,46.475,VILLA,GOOD,1849.0,2,100.000000,4
...,...,...,...,...,...,...,...
10270,41.290,HOUSE,AS_NEW,1075.0,3,150.000000,4
10271,30.074,HOUSE,GOOD,2818.0,2,163.000000,3
10273,31.331,HOUSE,AS_NEW,1310.0,3,162.000000,3
10275,33.097,HOUSE,AS_NEW,327.0,3,126.000000,4


In [20]:
property_type = list(data_postcodes['Subtype of property'].unique())
building_state = list(data_postcodes['State of the building'].unique())
num_facades = list(data_postcodes['Number of facades'].sort_values().unique().astype('int32'))
num_rooms = list(range(data_postcodes['Number of rooms'].max() + 1))

In [28]:
type(num_rooms[1])

int

In [10]:
new_data = {'PostalCodes': ['1083'],
            'Subtype of property': ['HOUSE'],
            'State of the building': ['AS_NEW'],
            'Number of facades': [2],
            'Number of rooms': [3],
            'Surface area of the plot of land': [218.00],
            'Living Area': [96.00]}

In [11]:
new_data = pd.DataFrame(new_data)

In [12]:
new_data['PostalCodes'] = new_data['PostalCodes'].astype('str')
new_data['Mean_income_taxunit'] = taxdata[taxdata['PostalCodes'] == new_data['PostalCodes'][0]]['Mean_income_taxunit'].values
new_data.drop(columns= ['PostalCodes'], inplace= True)

In [13]:
new_data['Subtype of property'] = new_data['Subtype of property'].astype('str')
new_data['State of the building'] = new_data['State of the building'].astype('str')
new_data['Number of facades'] = new_data['Number of facades'].astype('int64')
new_data['Number of rooms'] = new_data['Number of rooms'].astype('int64')
new_data['Surface area of the plot of land'] = new_data['Surface area of the plot of land'].astype('float')
new_data['Living Area'] = new_data['Living Area'].astype('float')

In [14]:
training_indices = pickle.load(open('./preprocessing/training_indices.pkl', 'rb'))
selected_rows = data.iloc[training_indices]
y_train = data_y.iloc[training_indices]

In [18]:
data_to_dist = pd.concat([selected_rows, new_data], ignore_index=True)

In [20]:
gower_mat = gower_matrix(data_to_dist)
gower_mat[-1]

array([0.32628626, 0.37150842, 0.35199922, ..., 0.35091603, 0.35818812,
       0.        ], dtype=float32)

In [21]:
dist_data = gower_mat[-1, :-1].reshape(1, -1)

In [107]:
model = pickle.load(open('./model/best_knn_model_reduced.pkl', 'rb'))

In [119]:
pred_price = model.predict(dist_data)

In [133]:
pred_price[0][0].round(2)

301459.82

In [57]:
predictions = []
knn = KNeighborsRegressor(n_neighbors= 19, metric= 'precomputed', weights= 'distance')

In [ ]:
X_dist = gower_mat[:-1, :-1]

In [51]:
def resample_distance_matrix(old_matrix, distance_array, y_values):
    # Get the size of the matrix
    n = old_matrix.shape[0]

    # Generate the list of indices and resample with replacement
    indices = np.arange(n)
    resampled_indices = np.random.choice(indices, size=n, replace=True)

    # Construct a new matrix based on the resampled indices
    new_matrix = old_matrix[np.ix_(resampled_indices, resampled_indices)]

    distance_column = distance_array.reshape(-1, 1)
    new_distance_array = distance_column[resampled_indices]
    new_distance_array = new_distance_array.reshape(1, -1)
    
    new_y_array = y_values[resampled_indices]
    
    return new_matrix, new_distance_array, new_y_array

In [64]:
new_matrix, new_distance_array, new_y_array = resample_distance_matrix(X_dist, dist_data, y_train.values)

In [65]:
knn.fit(new_matrix, new_y_array)


KNeighborsRegressor(metric='precomputed', n_neighbors=19, weights='distance')

In [66]:
predictions.append(knn.predict(new_distance_array)[0])

In [67]:
predictions

[310122.228803158, 315209.5484848011]